## Описание

Для выполнения задания был написан 
а) парсер для сбора данных, похожих на данный тестовой выборки (отзывы на мобильные телефоны),
б) подобрана модель, показавшее наилучшее качество на тестовых данных.

Для парсинга использовалась библиотека Scrapy и сайт отзывов на товары mail.ru (Яндекс.Маркет упорно сопротивлялся парсингу, в результате удалось набрать только 1700 отзывов, причем с сильным перевесом в пользу положительных). С сайта 
https://torg.mail.ru было собрано почти 20 000 отзывов, из них около 4 000 негативных. Чтобы классы были сбалансированы, использовалось поровну положительных и отрицательных отзывов, общий размер выборки около 8 000.


### Парсер

Ниже приводится класс парсера для https://torg.mail.ru. В качестве положительных отзывов брался текст блоков "Комментарий" и "Достоинства" отзывов, чей рейтинг выше или равен 4.5. В качестве отрицательных - текст блоков "Комментарий" и "Недостатки" отзывов, чей рейтинг ниже или равен 3 (остальные отзывы пропускались).

In [ ]:
class ReviewsSpider(scrapy.Spider):
    name = "reviews"
    POS = 'positive'
    NEG = 'negative'
    start_urls = [
        'https://torg.mail.ru/review/goods/mobilephones/?page=723',
    ]
    def __init__(self):
        self.counters = {self.POS: 0,
                         self.NEG: 0
                        };
        self.max_reviews = 8000;


    def parse(self, response):
        logging.info('response:%s\n', response)

        links = response.css('div.card__responses__good_information__name a::attr("href")')
        for link in links:
            logging.info('Link:%s\n', link.extract())
            yield scrapy.Request(link.extract(), self.unwrap_reviews)

        # follow pagination links
        pager = response.css('div.pager')
        if len(pager) > 0:
            fwd = pager.xpath('.//a[@title="%s"]/@href' % u'Следующая страница').extract_first()
            logging.info(self.counters)
            if fwd is not None and any(value < self.max_reviews for value in self.counters.values()):
                logging.info('Next product page:%s\n', fwd)
                yield response.follow(fwd, self.parse)

    def unwrap_reviews(self, response):
        # unwrap all reviews
        unwrap_link = response.xpath('.//div[@class="content_grid__center_column__box"]/div[contains(@class, "controls-row")]//a/@href').extract_first()
        logging.info('Unwrap Link:%s\n', unwrap_link)
        if unwrap_link is not None:
            logging.info('Request parse_page')
            yield scrapy.Request(unwrap_link, self.parse_page)
        else:
            yield self.parse_page(response)

    def parse_page(self, response):
        logging.info('parse_page %s' % response)
        logging.info(self.counters)
        reviews = response.css('div.review-item')
        n_reviews = len(reviews.extract())
        logging.info('Retrieved %i reviews', n_reviews)
        if n_reviews ==0:
            logging.info(response)

        for r in reviews:
            id_link = r.xpath('.//div[@class="review-item__publication-info"]/a/@href').extract_first()
            id_re = re.search(r'review_id=(\d+)', id_link)
            id = int(id_re.group(1))
            rating = float(r.xpath('.//span[@class="review-item__rating-counter"]/text()').extract_first().replace(',', '.'))
            logging.info('ID=%i\n', id)
            logging.info('rating=%i\n', rating)
            item = None
            if rating >= 4.5:
                text = self.get_text(r, self.POS)
                if text is None:
                    continue

                self.counters[self.POS] += 1;
                logging.info('positive counter=%i\n', self.counters[self.POS])
                item = {'y': 1,
                        'id': id,
                        'text': text.replace('\n', ' ')}
                yield item
            elif rating <= 3:
                text = self.get_text(r, self.NEG)
                if text is None:
                    continue

                self.counters[self.NEG] += 1;
                logging.info('negative counter=%i\n', self.counters[self.NEG])
                item = {'y': 0,
                        'id': id,
                        'text': text.replace('\n', ' ')}
                yield item

        # follow pagination links
        pager = response.css('div.pager')
        if len(pager) > 0:
            fwd = pager.xpath('.//a[@title="%s"]/@href' % u'Следующая страница').extract_first()
            logging.info(self.counters)
            if fwd is not None and any(value < self.max_reviews for value in self.counters.values()):
                logging.info('Next review page:%s\n', fwd)
                yield response.follow(fwd, self.parse_page)

    def get_text(self, review, sentiment):
        comment = review.xpath('.//div[@class="review-item__content"]/p[1]//span/text()').extract_first()
        comment_text = comment if comment is not None else ''
        if sentiment == self.POS:
            pos_comment = review.xpath('.//div[@class="review-item__content"]/div[text()="%s"]/following-sibling::p[1]//span/text()' % u'Достоинства').extract_first()
            pos_comment_text = pos_comment if pos_comment is not None else ''
            text = comment_text.strip() + \
                   pos_comment_text.strip()
        elif sentiment == self.NEG:
            neg_comment = review.xpath('.//div[@class="review-item__content"]/div[text()="%s"]/following-sibling::p[1]//span/text()' % u'Недостатки').extract_first()
            neg_comment_text = neg_comment if neg_comment is not None else ''
            text = comment_text.strip() + \
                   neg_comment_text.strip()
        else:
            raise Exception('Unknown sentiment %s', sentiment)
        if text is None or len(text) < 1:
            return None
        return text

### Модель

На обучающей выборке наилучшее качество показали классификаторы SGDClassifier, LogisticRegression и MLPClassifier. Для векторизации использовался TfidfVectorizer. 
Также была попытка применить предварительную лемматизацию, замену слов синонимами для уменьшения словаря и добавление маркеров эмоциональной окраски (Positive/Negative).
На обучающей выборке это дало более высокое качество (87%), на тестовой - такое же, как и TfidfVectorizer + MLPClassifier без предобрботки (98%).

In [6]:
import random
import os
from xml.dom import minidom

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV

import pandas as pd
import numpy as np

### Загрузка данных

In [7]:
# Загрузка данных для обучения
def load_reviews(path, sep=',', shuffle=True, balance = True):
    df = pd.read_csv(path, sep=sep, encoding ='utf8')
    df = df.dropna()
    df = df.drop(df[df['text'].map(len) < 10].index)

    if balance:
        g = df.groupby('y')
        df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

    df.ix[:]['text'].apply(lambda x: x.replace('\n', ' '))
    X_train = df.ix[:]['text'].values
    y_train = df.ix[:]['y'].values
    Xy = zip(X_train, y_train)
    random.shuffle(Xy, lambda: random.random() if shuffle else 0.42)
    return [x[0] for x in Xy], [x[1] for x in Xy]

# Загрузка тестовых данных
def load_reviews_xml(path):
    path = os.path.join(os.path.dirname(os.path.abspath(__file__)), path)
    xmldoc = minidom.parse(path)
    itemlist = xmldoc.getElementsByTagName('review')
    reviews = [item.firstChild.nodeValue.encode('utf-8').decode('utf-8') for item in itemlist]
    df = pd.DataFrame(data={'text': reviews, 'Id': range(0, len(reviews))})
    return df

### Вспомогательные функции

In [8]:
def show_pipeline_results(pipeline, accuracy):
    print('{}\nAccuracy={:f}\n'.format(pipeline.named_steps.keys(), accuracy))
    
def write_predictions(fname, df):
    df.to_csv(fname, columns=['Id', 'y'], index=False)
    
def get_pipeline_predictions(pipeline, X_train, y_train, X_test):
    labels = LabelEncoder()
    y_train = labels.fit_transform(y_train)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return y_pred

def evaluate(clf, X, y, cv=3):
    scores = cross_val_score(clf, X, y, cv=cv)
    return scores.mean()

# Подбор параметров по сетке
def grid_search(clf, params_grid, X, y):
    gs = GridSearchCV(clf, params_grid)
    gs.fit(X, y)
    return gs

In [9]:
X_train, y_train = load_reviews('data/reviews_train.csv', balance=True)


### Модели

In [11]:
def test_pipelines_params2(X, y):
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=0, max_df=0.9, norm='l2',ngram_range=(1,2), stop_words='english', max_features=None)),
        ('clf', SGDClassifier(penalty='l2', loss='hinge', alpha=1e-5, n_iter=50)),
    ])
    #('Best params: ', {'tfidf__stop_words': 'english', 'clf__loss': 'hinge', 'tfidf__ngram_range': (1, 2), 'tfidf__max_df': 0.75, 'clf__penalty': 'l2',
    # 'clf__alpha': 1e-05, 'clf__n_iter': 50}) Best score: 0.867101
    params_grid = {#'tfidf__ngram_range': ((1,2), (1,3)),
                   #'tfidf__min_df': (0, 1),
                   #'tfidf__max_df': (0.75, 0.9, 1),
                   #'tfidf__max_features': (1000, 5000, 10000, None),
                   #'tfidf__stop_words' : ('english', None),
                   #'tfidf__norm': ('l1', 'l2', None),
                   'clf__alpha': (1e-4, 1e-5, 1e-6),
                   'clf__penalty': ('l1','l2', 'elasticnet'),
                   'clf__loss': ('hinge','log', 'perceptron'),
                   #'clf__n_iter': (10, 50, 80),
                   }
    res = grid_search(pipe, params_grid, X, y)
    print ('Best params: ', res.best_params_)
    print 'Best score: %f' % res.best_score_
    return pipe


def test_pipelines_params3(X, y):
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=0, max_df=0.75, norm='l2',ngram_range=(1,2), stop_words='english')),
        ('clf', LogisticRegression(solver='newton-cg', C=150, tol=0.9)),
    ])
    #'Best params: ', {'tfidf__stop_words': 'english', 'tfidf__min_df': 0, 'tfidf__max_features': None, 'clf__tol': 0.9, 
    #'clf__C': 150, 'clf__solver': 'newton-cg'}) Best score: 0.868066
    params_grid = {#'tfidf__ngram_range': ((1,2), (1,3)),
                   #'tfidf__min_df': (0, 2, 10),
                   #'tfidf__max_df': (0.75, 0.9, 1),
                   #'tfidf__max_features': (5000, 10000, None),
                   #'tfidf__stop_words' : ('english', None),
                   #'tfidf__norm': ('l1', 'l2', None),
                   'clf__C': (100, 150, 180),
                   'clf__solver': ('newton-cg', 'liblinear', 'sag'),
                   #'clf__penalty': ('l2'),
                   'clf__tol': (1, 0.9, 0.75),
                   #'clf__max_iter': (10, 50, 80),
                   }
    res = grid_search(pipe, params_grid, X, y)
    print ('Best params: ', res.best_params_)
    print 'Best score: %f' % res.best_score_
    return pipe

    
def test_pipelines_params4(X, y):
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=0, max_df=0.9, norm='l2',ngram_range=(1,2), stop_words=None, max_features=None)),
        ('clf', MLPClassifier(hidden_layer_sizes=(30), solver='sgd', activation='tanh', learning_rate_init=0.1, learning_rate='constant', momentum=0.8)),
        #'Best params: ', {'tfidf__max_df': 0.9, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': None, 
        # 'tfidf__max_features': None, 'tfidf__norm': 'l2', 'clf__solver': 'sgd'
        # 'clf__momentum': 0.8, 'clf__learning_rate': 'constant'}) Best score: 0.869513
    ])
    params_grid = {'tfidf__ngram_range': ((1,2), (1,3)),
                   #'tfidf__min_df': (0, 2, 10),
                   'tfidf__max_df': (0.75, 0.9, 1),
                   'tfidf__max_features': (5000, 10000, None),
                   #'tfidf__stop_words' : ('english', None),
                   #'tfidf__norm': ('l1', 'l2', None),
                   #'clf__activation': ('logistic', 'tanh', 'relu'),
                   #'clf__solver': ('lbfgs', 'sgd', 'adam'),
                   #'clf__alpha': (0.01, 0.001, 0.0001),
                   #'clf__learning_rate': ('constant', 'adaptive'),
                   #'clf__learning_rate_init': (0.5, 0.1, 0.01),
                   #'clf__max_iter': (100, 200, 500),
                   #'clf__momentum': (0.8, 0.9, 1)
                   #'clf__hidden_layer_sizes': ((30), (100), (70, 30))
                   }
    res = grid_search(pipe, params_grid, X, y)
    print ('Best params: ', res.best_params_)
    print 'Best score: %f' % res.best_score_
    return pipe

In [ ]:
test_pipelines_params2(X_train, y_train)
test_pipelines_params3(X_train, y_train)
pipeline = test_pipelines_params4(X_train, y_train)

('Best params: ', {'clf__penalty': 'l2', 'clf__loss': 'log', 'clf__alpha': 1e-06})
Best score: 0.867463
('Best params: ', {'clf__tol': 0.75, 'clf__C': 100, 'clf__solver': 'newton-cg'})
Best score: 0.868066


Загрузка тестовых данных:

In [ ]:
df_test = load_reviews_xml('data/reviews_test.xml')
X_test = df_test.ix[:]['text'].values

Предсказание:

In [ ]:
y_test = get_pipeline_predictions(pipeline, X_train, y_train, X_test)
df_test['y'] = ['neg' if y == 0 else 'pos' for y in y_test]
write_predictions('data/phone_reviews_sentiment_prediction.csv', df_test)

На тестовой выборке качество модели 2 - 96%, моделей 3 и 4 - 98%.

Можно сделать вывод, что основную роль в данной задаче сыграл объем обучающей выборки, подбор классификатора и настройка параметров дали небольшое увеличение качества, обработка данных не дала заметного увеличения качества модели.